In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

class MyDataset2(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data


def val_missing_data_rbf(df, real, rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    real_masks = ~np.isnan(real.values)
    
    masks = masks.reshape(shp)
    real_masks = real_masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    real = torch.nan_to_num(torch.from_numpy(real.values).to(torch.float32))

    masks = torch.from_numpy(masks).to(torch.float32)
    real_masks = torch.from_numpy(real_masks).to(torch.float32)
    
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)


    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0), real.unsqueeze_(0), real_masks.unsqueeze_(0)], dim = 0)
    mydata  = MyDataset2(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)
    
    return data

def eval_model(model, rbf, realpath, dfpath, batch_size, seq_len):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)

    dataset = val_missing_data_rbf(df,real, rbf, batch_size, seq_len)

    model.eval()
    c_list = []
    train_list = []
    real_list = []
    real_masks_list = []
    masks_list = []

    batch_x_loss = 0.0
    # batch_loss = 0.0
    # batch_d_loss = 0.0

    for data in dataset:
        data = data.to(device)
        value = data[:,0,:].clone().detach()
        real_value = data[:,4,:].clone().detach()
        real_masks = data[:,5,:].clone().detach()
        train_masks = data[:,2,:].clone().detach()

        train_list.append(value)
        real_list.append(real_value)
        masks_list.append(train_masks)
        real_masks_list.append(real_masks)

        c_hat_list, imputations, x_loss = model(data)
        c_list.append(c_hat_list)
        batch_x_loss += x_loss
        # batch_d_loss += D_loss
        # batch_loss = loss

    c_list = torch.cat(c_list, dim = 0)
    c_list_ori = c_list * torch.tensor(df.std().values, device= device).reshape(1,1,36) + torch.tensor(df.mean().values, device= device).reshape(1,1,36)

    real_list = torch.cat(real_list, dim = 0) # none scale
    real_masks_list = torch.cat(real_masks_list, dim = 0)

    masks_list = torch.cat(masks_list, dim = 0)
    train_list = torch.cat(train_list, dim = 0)

    missing_masks = real_masks_list - masks_list
    # print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    # print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(missing_masks))
    print("Original MRE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(torch.abs(real_list * missing_masks)))
    print()
    print('Train MAE :', torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(masks_list))
    print("Train MRE :", torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(torch.abs(real_list * masks_list)))
    print('--------------------------------------------------------------')


    print("x_loss: {}".format(batch_x_loss))

def val_missing_data_rbf2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model2(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_los = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [2]:
def eval_model_ver2(model, rbf, realpath, dfpath, batch_size, seq_len):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)

    dataset = val_missing_data_rbf(df,real, rbf, batch_size, seq_len)

    model.eval()
    c_list = []
    train_list = []
    real_list = []
    real_masks_list = []
    masks_list = []

    batch_x_loss = 0.0
    # batch_loss = 0.0
    # batch_d_loss = 0.0

    for data in dataset:
        data = data.to(device)
        value = data[:,0,:].clone().detach()
        real_value = data[:,4,:].clone().detach()
        real_masks = data[:,5,:].clone().detach()
        train_masks = data[:,2,:].clone().detach()

        train_list.append(value)
        real_list.append(real_value)
        masks_list.append(train_masks)
        real_masks_list.append(real_masks)

        c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_losss = model(data)
        c_list.append(c_hat_list)
        batch_x_loss += x_loss
        # batch_d_loss += D_loss
        # batch_loss = loss

    c_list = torch.cat(c_list, dim = 0)
    c_list_ori = c_list * torch.tensor(df.std().values, device= device).reshape(1,1,36) + torch.tensor(df.mean().values, device= device).reshape(1,1,36)

    real_list = torch.cat(real_list, dim = 0) # none scale
    real_masks_list = torch.cat(real_masks_list, dim = 0)

    masks_list = torch.cat(masks_list, dim = 0)
    train_list = torch.cat(train_list, dim = 0)

    missing_masks = real_masks_list - masks_list
    # print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    # print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(missing_masks))
    print("Original MRE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(torch.abs(real_list * missing_masks)))
    print()
    print('Train MAE :', torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(masks_list))
    print("Train MRE :", torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(torch.abs(real_list * masks_list)))
    print('--------------------------------------------------------------')


    print("x_loss: {}".format(batch_x_loss))

def val_missing_data_rbf2_ver2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model2_ver2(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2_ver2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_losss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

In [24]:
class MGRU_ver3(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU_ver3, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            

            r_hat = self.temp_decay_r(r)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            RG = torch.cat([x_hat, r_hat], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(x_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))

            hid = hid * gamma_h
            
            # GRU cell
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss

In [22]:
def train_MGRU_ver2(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_x_hat_loss = 0.0
        batch_concat_loss = 0.0
        batch_z_hat_loss = 0.0
        batch_c_hat_loss = 0.0
        for data in dataset:
            data = data.to(device)
            c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
            batch_x_hat_loss += x_hat_loss
            batch_concat_loss += concat_loss
            batch_z_hat_loss += z_hat_loss
            batch_c_hat_loss += c_hat_loss

        progress.set_description("loss: {},x_hat : {}, concat_hat : {}, z_hat : {}, c_hat : {}".format(batch_loss, batch_x_hat_loss, batch_concat_loss, batch_z_hat_loss, batch_c_hat_loss))

    return x_loss

In [18]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset3 = missing_data_rbf(df, rbfpath, batch_size, 36)

In [20]:
model_ver1_ver2 = MGRU_ver5(36, 64)
model_ver1_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver1_ver2, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 56.18926239013672,x_hat : 20.16445541381836, concat_hat : 5.224947929382324, z_hat : 25.590578079223633, c_hat : 5.209273338317871: 100%|██████████| 1000/1000 [12:29<00:00,  1.33it/s]  


Original MAE : tensor(16.1537, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2271, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(2.8109, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.0332, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 55.73569869995117
------------------------------------------------------
Scale MAE : tensor(0.1873)
Scale MRE : tensor(0.2703)
Original MAE : 15.168766408974058
Original MRE : 0.2130733119602795
Train MAE : 2.7875853849149412
Train MRE : 0.03288228943486652


In [26]:
model_ver1_ver2 = MGRU_ver5(36, 64)
model_ver1_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver1_ver2, 0.001, 500, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 62.58552551269531,x_hat : 21.206974029541016, concat_hat : 7.862831115722656, z_hat : 25.661972045898438, c_hat : 7.853741645812988: 100%|██████████| 500/500 [05:33<00:00,  1.50it/s]  


Original MAE : tensor(16.8925, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2375, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(4.1298, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.0487, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.92649841308594
------------------------------------------------------
Scale MAE : tensor(0.1960)
Scale MRE : tensor(0.2829)
Original MAE : 15.870516027393316
Original MRE : 0.22293068014250647
Train MAE : 4.095647949087113
Train MRE : 0.04831216364312704
